# PostgreSQL Vector Db Demo

**Refs**:
  * [Installing PostgreSQL on Arch](https://wiki.archlinux.org/title/PostgreSQL)
  * [PgVector](https://github.com/pgvector/pgvector)
  * [Python Client for PgVector](https://github.com/pgvector/pgvector-python)

1. Install postgresql and ensure it is running.
2. Install pgvector (probably will need to make it from source).
3. Install pgvector-python (`pip install pgvector`).
4. Install the pgvector extension in everydatabase I want to use it by running the following once in the database -
```sql
CREATE EXTENSION vector;
```

In [1]:
import psycopg as pg
from pgvector.psycopg import register_vector
import numpy as np

In [2]:
embedding = np.array([100,200,300])

Ensure that the database `vectordb` has the pgvector extension installed.

In [3]:
with pg.connect("dbname=vectordb user=pguser") as conn:
    register_vector(conn)
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM items ORDER BY embedding <-> %s LIMIT 2", (embedding,))
        for row in cur.fetchall():
            print(row)

(3, array([100., 200., 300.], dtype=float32))
(2, array([4., 5., 6.], dtype=float32))


In [4]:
conn.closed

True

In [5]:
import os
"OPENAI_API_KEY" in os.environ

True

In [6]:
from openai import OpenAI

In [7]:
client = OpenAI()

In [8]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Functions call themselves,  \nDepths of logic intertwine,  \nEndless paths converge.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [9]:
completion.choices[0].message

ChatCompletionMessage(content='Functions call themselves,  \nDepths of logic intertwine,  \nEndless paths converge.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

In [10]:
from sentence_transformers import SentenceTransformer

In [11]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [12]:
sentences = [
    "The new Finder puts a user 's folders , hard drive , network servers , iDisk and removable media in one location , providing one-click access .",
    "Panther 's redesigned Finder navigation tool puts a user 's favourite folders , hard drive , network servers , iDisk and removable media in one location .",
    "But tropical storm warnings and watches were posted today for Haiti , western portions of the Dominican Republic , the southeastern Bahamas and the Turk and Caicos islands .",
    "Tropical storm warnings were in place Thursday for Jamaica and Haiti and watches for the western Dominican Republic , the southeastern Bahamas and the Turks and Caicos islands .",
    "Singapore is already the United States ' 12th-largest trading partner , with two-way trade totaling more than $ 34 billion .",
    "Although a small city-state , Singapore is the 12th-largest trading partner of the United States , with trade volume of $ 33.4 billion last year ."
]

In [13]:
embs = [model.encode(sentence) for sentence in sentences]

In [14]:
embs[0].shape

(384,)

In [15]:
insert_sql = "INSERT INTO sentence_xformers (embedding, content) VALUES (%s, %s)"

In [16]:
with pg.connect("dbname=pgv_quickstart user=pguser") as conn:
    register_vector(conn)
    with conn.cursor() as cur:
        for i in range(len(sentences)):
            conn.execute(insert_sql, (embs[i], sentences[i]))

In [17]:
query_sentence = "The dynamic economic partnership between Singapore and the United States has resulted in mutual trade volume exceeding $34 billion, with Singapore emerging as the U.S.' 12th-largest trading partner."
query_emb = model.encode(query_sentence)

In [18]:
query_emb.shape

(384,)

In [19]:
type(query_emb)

numpy.ndarray

In [20]:
type(embs[0])

numpy.ndarray

In [21]:
embs[0].shape

(384,)

In [22]:
from sklearn.metrics import pairwise_distances

In [23]:
for sentence, emb in zip(sentences, embs):
    l2 = pairwise_distances(emb.reshape(1, -1), query_emb.reshape(1, -1))
    print(f"l2 = {l2}")
    print(sentence)
    print("\n")


l2 = [[1.4468412]]
The new Finder puts a user 's folders , hard drive , network servers , iDisk and removable media in one location , providing one-click access .


l2 = [[1.3744501]]
Panther 's redesigned Finder navigation tool puts a user 's favourite folders , hard drive , network servers , iDisk and removable media in one location .


l2 = [[1.4506443]]
But tropical storm warnings and watches were posted today for Haiti , western portions of the Dominican Republic , the southeastern Bahamas and the Turk and Caicos islands .


l2 = [[1.444849]]
Tropical storm warnings were in place Thursday for Jamaica and Haiti and watches for the western Dominican Republic , the southeastern Bahamas and the Turks and Caicos islands .


l2 = [[0.56965816]]
Singapore is already the United States ' 12th-largest trading partner , with two-way trade totaling more than $ 34 billion .


l2 = [[0.53106487]]
Although a small city-state , Singapore is the 12th-largest trading partner of the United States , 

### Distance Functions
PgVector supports the following distance functions, each with its own weird symbol -
* `<->` - L2 distance
* `<#>` - (negative) inner product
* `<=>` - cosine distance
* `<+>` - L1 distance (added in 0.7.0)
* `<~>` - Hamming distance (binary vectors, added in 0.7.0)
* `<%>` - Jaccard distance (binary vectors, added in 0.7.0)

In [24]:
select_sql = "SELECT id, content, embedding <-> %s AS distance FROM sentence_xformers"
# select_sql = "select id, content from sentence_xformers"

with pg.connect("dbname=pgv_quickstart user=pguser") as conn:
    register_vector(conn)
    with conn.cursor() as cur:
        cur.execute(select_sql, (query_emb,))
        # cur.execute(select_sql)
        for row in cur.fetchall():
            print(row)

(1, "The new Finder puts a user 's folders , hard drive , network servers , iDisk and removable media in one location , providing one-click access .", 1.4468411995748218)
(2, "Panther 's redesigned Finder navigation tool puts a user 's favourite folders , hard drive , network servers , iDisk and removable media in one location .", 1.3744500534291297)
(3, 'But tropical storm warnings and watches were posted today for Haiti , western portions of the Dominican Republic , the southeastern Bahamas and the Turk and Caicos islands .', 1.4506442316005759)
(4, 'Tropical storm warnings were in place Thursday for Jamaica and Haiti and watches for the western Dominican Republic , the southeastern Bahamas and the Turks and Caicos islands .', 1.4448490533889198)
(5, "Singapore is already the United States ' 12th-largest trading partner , with two-way trade totaling more than $ 34 billion .", 0.569658165331814)
(6, 'Although a small city-state , Singapore is the 12th-largest trading partner of the Un

In [25]:
select_sql = "SELECT id, content FROM sentence_xformers ORDER BY embedding <=> %s LIMIT 2"

with pg.connect("dbname=pgv_quickstart user=pguser") as conn:
    register_vector(conn)
    with conn.cursor() as cur:
        cur.execute(select_sql, (query_emb,))
        for row in cur.fetchall():
            print(row)

(6, 'Although a small city-state , Singapore is the 12th-largest trading partner of the United States , with trade volume of $ 33.4 billion last year .')
(12, 'Although a small city-state , Singapore is the 12th-largest trading partner of the United States , with trade volume of $ 33.4 billion last year .')
